## Imports

In [ ]:
import subprocess
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# import pgeocode
# import ssl

## Connecting to Database

In [ ]:
DB_Password = subprocess.run(["security", "find-generic-password", 
 "-w", "-a", "datalab", "-s" "postgres"], capture_output=True, text=True).stdout.strip("\n")

In [ ]:
# Use this line for the left-most computer with the database on it.
engine = create_engine('postgresql://postgres:{}@localhost:5432/PublicHealth'.format(DB_Password))

# Use this line for the other computers.
# engine = create_engine('postgresql://postgres:{}@10.97.127.134:5432/PublicHealth'.format(DB_Password))

In [ ]:
connection = engine.connect()

In [ ]:
%%time
query='''
select sewanee_health_data."File_Type" , sewanee_health_data."Data_Yr", sewanee_health_data."Hospital_ID", sewanee_health_data."Patient_Zip", sewanee_health_data."Patient_Sex", sewanee_health_data."Admit_Hr", sewanee_health_data."Admission_Type", sewanee_health_data."Admission_Source", sewanee_health_data."Accident_Code", sewanee_health_data."Primary_Health_Plan_Id",
sewanee_health_data."Diag1", sewanee_health_data."Diag2", sewanee_health_data."Diag3", sewanee_health_data."Diag4", sewanee_health_data."Diag5", sewanee_health_data."Diag6", sewanee_health_data."Diag7", sewanee_health_data."Diag8", sewanee_health_data."Diag9", sewanee_health_data."Diag10", sewanee_health_data."Diag11", sewanee_health_data."Diag12", sewanee_health_data."Diag13", sewanee_health_data."Diag14", sewanee_health_data."Diag5", sewanee_health_data."Diag16", sewanee_health_data."Diag17", sewanee_health_data."Diag18", 
sewanee_health_data."Patient_Reason_Visit1", sewanee_health_data."Patient_Reason_Visit2", sewanee_health_data."Patient_Reason_Visit3", sewanee_health_data."Patient_Race_Ethnicity", sewanee_health_data."Type_ER_Visit", sewanee_health_data."Patient_ID", sewanee_health_data."Age", sewanee_health_data."Hospital_Id_JAR", sewanee_health_data."Primary_Payer_Class_Cd"
 from sewanee_health_data
where sewanee_health_data."Patient_Zip" in ('37342','37349','37355','37382', '37388','37389','37306','37318','37324','37330','37345','37372','37375','37376', '37398','37352')
'''

#Data with select set of 16 zip codes
#df = pd.read_sql(query, connection)

In [ ]:
df = pd.read_sql(query, connection)

## Exploring Data

In [ ]:
df.head()

In [ ]:
df.info

In [ ]:
df.describe()

df.sum()isnull()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
print(df['Diag1'].value_counts())

In [ ]:
df.loc[[200,250,3000,55000,77456,80000]]

In [ ]:
print(df['Primary_Health_Plan_Id'].value_counts())

In [ ]:
df.Primary_Health_Plan_Id.unique()

In [ ]:
df.Primary_Health_Plan_Id.nunique()

In [ ]:
(df.Primary_Health_Plan_Id == 'SELF PAY NO INS').sum()

In [ ]:
(df.Primary_Health_Plan_Id == 'SELF PAY').sum()

In [ ]:
(df.Primary_Health_Plan_Id == 'None').sum()

In [ ]:
(df.Primary_Health_Plan_Id == 'SelfPay').sum()

In [ ]:
(df.Primary_Health_Plan_Id == '999999999999999').sum()

In [ ]:
df.Primary_Payer_Class_Cd.unique()

In [ ]:
(df.Primary_Payer_Class_Cd=='P').sum()
#
#P= self pay

In [ ]:
(df.Primary_Payer_Class_Cd=='O').sum()
# o=other

In [ ]:
df.Hospital_ID.unique()

In [ ]:
df.Hospital_Id_JAR.unique()

In [ ]:
print(df['Patient_Zip'].value_counts())

In [ ]:
df.Hospital_ID.nunique()

In [ ]:
df.Type_ER_Visit.unique()

In [ ]:
df.Type_ER_Visit.nunique()

In [ ]:
df.Accident_Code.unique()

In [ ]:
print(df['Accident_Code'].value_counts())

In [ ]:
print(df['Type_ER_Visit'].value_counts())

In [ ]:
print(df['Hospital_ID'].value_counts())

In [ ]:
print(df['Diag1'].value_counts())

In [ ]:
print(df['Patient_Reason_Visit1'].value_counts())

In [ ]:
#Checking if accident code has non-null values
df_acodes = df[df['Accident_Code'].isnull() == False]
df_acodes

In [ ]:
#Checking if there are patients with 18 diagnoses
df_acodes = df[df['Diag18'].isnull() == False]
df_acodes

In [ ]:
df_acodes = df[df['Diag1'].isnull() == True]
df_acodes

## Data Cleaning

In [ ]:
#Combine Diag2-Diag18 into one column Extra_Diags
df['Extra_Diags'] = df[df.columns[11:28]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

In [ ]:
df.head()

In [ ]:
#View all column names
for col in df.columns:
        print(col)

In [ ]:
#Combine Patient_Reason_Visit2-3 into one column Extra_Reasons
df['Extra_Reasons'] = df[df.columns[29:31]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df.head()

In [ ]:
df[df['Patient_Reason_Visit1'].notnull()][['Patient_Reason_Visit1',
'Patient_Reason_Visit2',
'Patient_Reason_Visit3',
'Patient_Race_Ethnicity',
'Type_ER_Visit']]

In [ ]:
df[df['Primary_Payer_Class_Cd']=='P'].shape

In [ ]:
df.shape

In [ ]:
#Dropping Diag2-18
df.drop(['Diag2','Diag3','Diag4','Diag5','Diag6','Diag7','Diag8','Diag9','Diag10','Diag11',
        'Diag12','Diag13','Diag14','Diag5','Diag16','Diag17','Diag18'],axis=1,inplace=True)
for col in df.columns:
        print(col)

In [ ]:
#Dropping Patient Reason2-3
df.drop(['Patient_Reason_Visit2','Patient_Reason_Visit3'],axis=1,inplace=True)
for col in df.columns:
        print(col)

In [ ]:
#Dropped 3 columns where Diag1 was null
df.drop([41590,41822,77843],inplace=True)

In [ ]:
df['Patient_Reason_Visit1'].fillna("Unspecified", inplace=True)

In [ ]:
df.dtypes

In [ ]:
df1 = df.groupby('Diag1')
df1.head()

## Graphs using limited dataset

In [ ]:
# Get the value counts of each diagnosis
diagnosis_counts = df['Diag1'].value_counts()
# Select the top 10 diagnoses
top_10_diagnoses = diagnosis_counts.head(10)
# Create a bar plot of the top 10 diagnoses
plt.figure(figsize=(10, 6))
top_10_diagnoses.plot(kind='bar')
plt.title('Top 10 Diagnoses')
plt.xlabel('Diagnosis')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Get the value counts of each diagnosis
diagnosis_counts = df['Diag1'].value_counts()

# Select the top 10 diagnoses
top_10_diagnoses = diagnosis_counts.head(10)

# Create a DataFrame for the top 10 diagnoses
data = pd.DataFrame({'Diagnosis': top_10_diagnoses.index, 'Count': top_10_diagnoses.values})

# Create an interactive histogram using Plotly
fig = px.bar(data, x='Diagnosis', y='Count', title='Top 10 Diagnoses')
fig.update_layout(xaxis={'categoryorder':'total descending'})

# Display the interactive histogram
fig.show()

In [ ]:

top_10_diagnoses = df['Diag1'].value_counts().nlargest(10)
# Dictionary mapping ICD codes to diagnosis names
icd_to_diagnosis = {
    "N390": "Urinary tract infection, site not specified",
    "R109": "Unspecified abdominal pain",
    "R0789": "Other chest pain",
    "R079": "Chest pain unspecified",
    "M545": "Low back pain",
    "J069": "Acute upper respiratory infection, unspecified",
    "R112": "Nausea with vomiting, unspecified",
    "R51": "Headache",
    "J209": "Acute bronchitis, unspecified",
    "A419": "Sepsis unspecified organism"
}
# Mapping diagnosis names to the top 10 diagnoses
top_10_diagnoses_names = top_10_diagnoses.index.map(icd_to_diagnosis)
# Creating the bar graph using Plotly
fig = px.bar(x=top_10_diagnoses_names, y=top_10_diagnoses.values, labels={'x': 'Diagnosis', 'y': 'Count'})
fig.update_layout(title="Top 10 Diagnoses",
                  xaxis_title="Diagnosis",
                  yaxis_title="Count")
fig.show()

In [ ]:

top_10_diagnoses = df['Diag1'].value_counts().nlargest(10)
# Dictionary mapping ICD codes to diagnosis names
icd_to_diagnosis = {
    "N390": "UTI",
    "R109": "Abdominal pain",
    "R0789": "Other chest pain",
    "R079": "Chest pain",
    "M545": "Low back pain",
    "J069": "Acute URI",
    "R112": "Nausea with vomiting",
    "R51": "Headache",
    "J209": "Acute bronchitis",
    "A419": "Sepsis"
}
# Mapping diagnosis names to the top 10 diagnoses
top_10_diagnoses_names = top_10_diagnoses.index.map(icd_to_diagnosis)
# Creating the bar graph using Plotly
fig = px.bar(x=top_10_diagnoses_names, y=top_10_diagnoses.values, labels={'x': 'Diagnosis', 'y': 'Count'})
fig.update_layout(title="Top 10 Diagnoses",
                  xaxis_title="Diagnosis",
                  yaxis_title="Count")
fig.show()

In [ ]:
#Dictionary of top 10 ICD codes
ICD10_legend = {"N390":"Urinary tract infection, site not specified","R109":"Unspecified abdominal pain", 
                "R0789":"Other chest pain", "R079":"Chest pain unspecified","M545":"Low back pain",
               "J069":"Acute upper respiratory infection, unspecified","R112":"Nausea with vomiting, unspecified",
               "R51":"Headache","J209":"Acute bronchitis, unspecified","A419":"Sepsis unspecified organism"}
print(ICD10_legend)

In [ ]:
df.Patient_Zip.unique()

In [ ]:
df.Diag1.unique()

In [ ]:
#Dropping columns for the graph below so that no zip codes contain below 11 people

#conditions = (df['Patient_Zip'] == 37382) & df['Diag1'].isin(['I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19'])
#df = df[~conditions]

In [ ]:
# Specify the preventable diagnoses
preventable_diagnoses = ['I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19']
# Filter the DataFrame based on the preventable diagnoses
filtered_data = df[df['Diag1'].isin(preventable_diagnoses)]
# Create a histogram of Patient_Zip counts for the preventable diagnoses
fig = px.histogram(filtered_data, x='Patient_Zip', title='Count of Preventable Diagnoses by Zip Code')
# Display the interactive histogram
fig.show()

In [ ]:
#THIS IS THE CORRECT ONE

# Specify the preventable diagnoses
preventable_diagnoses = ['I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19']
# Filter the DataFrame based on the preventable diagnoses
filtered_data = df[df['Diag1'].isin(preventable_diagnoses)]
# Combine specified zip codes into one category 'Other'
filtered_data.loc[filtered_data['Patient_Zip'].isin(['37376', '37382', '37349']), 'Patient_Zip'] = 'Other'
# Create a histogram of Patient_Zip counts for the preventable diagnoses
fig = px.histogram(filtered_data, x='Patient_Zip', title='Count of Preventable Diagnoses by Zip Code')
# Display the interactive histogram
fig.show()

In [ ]:
#This one was testing if the count changes in an unexpected way
#when we include decimals in the ICD codes

# Specify the preventable diagnoses
preventable_diagnoses = ['I20', 'I24.0', 'I24.8', 'I24.9', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L98.0', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I11.0', 'J81.0', 'R56', 'E86', 'E10.9', 'E11.9', 'E10.1', 'E13.1', 'E11.0', 'E13.0', 'E10.641', 'E11.641', 'E10.6', 'E11.6', 'E10.8', 'E11.8', 'K52.9', 'K52.89', 'G40', 'I10', 'I11.9', 'E16.2', 'N10', 'N11', 'N12', 'N70', 'N73', 'A15.0', 'A15.5', 'A15.9', 'H66', 'J02', 'J03', 'J06', 'J31.2', 'A15.4', 'A15.6', 'A15.8', 'A17', 'A18', 'A19']
# Filter the DataFrame based on the preventable diagnoses
filtered_data = df[df['Diag1'].isin(preventable_diagnoses)]
# Create a histogram of Patient_Zip counts for the preventable diagnoses
fig = px.histogram(filtered_data, x='Patient_Zip', title='Count of Zip Codes for Preventable Diagnoses')
# Display the interactive histogram
fig.show()

In [ ]:
# Specify the zip codes and their respective counties
zip_county_mapping = {
    '37318': 'Franklin County',
    '37398': 'Franklin County',
    '37330': 'Franklin County',
    '37306': 'Franklin County',
    '37345': 'Franklin County',
    '37324': 'Franklin County',
    '37375': 'Franklin County',
    '37376': 'Franklin County',
    '37355': 'Coffee County',
    '37388': 'Coffee County',
    '37342': 'Coffee County',
    '37349': 'Coffee County',
    '37352': 'Moore County'
}
# Specify the preventable diagnoses
preventable_diagnoses = [
    'I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19'
]
# Filter the DataFrame based on zip codes and preventable diagnoses
filtered_data = df[df['Patient_Zip'].isin(zip_county_mapping.keys()) & df['Diag1'].isin(preventable_diagnoses)]
# Assign the respective counties to each zip code
filtered_data['County'] = filtered_data['Patient_Zip'].map(zip_county_mapping)
# Create an interactive histogram
fig = px.histogram(filtered_data, x='County', title='Count of Counties for Preventable Diagnoses')
# Display the interactive histogram
fig.show()

In [ ]:
# # CREATING THE DISTANCE COL
# ## description: a column that represents the distance between patients and the clinic (calculated by zip code)
# # function that gets the distance between two zip codes using the pgeocode package
# #def get_distance(x, y):
#     usa_zipcodes = pgeocode.GeoDistance('us')
#     distance_in_kms = usa_zipcodes.query_postal_code(x, y.values)
#     return distance_in_kms
# # creating the new column
# #df['Distance'] = get_distance('37388', df['Patient_Zip'])

In [ ]:
# # Set the SSL certificate verification path
# ssl._create_default_https_context = ssl._create_default_https_context._create_default_https_context

In [ ]:
df_acodes = df[df['Primary_Payer_Class_Cd'].isnull() == True]
df_acodes

In [ ]:
#Dropped 3 columns where Primary_Payer_Class_Cd was null
df.drop([75,38203,63230],inplace=True)

In [ ]:
df_acodes = df[df['Primary_Payer_Class_Cd'].isnull() == True]
df_acodes

In [ ]:

# Filter the dataframe for 'Primary_Payer_Class_Cd' equals 'P'
df_p = df[df['Primary_Payer_Class_Cd'] == 'P']
diagnosis_counts_p = df_p['Diag1'].value_counts().head(10)
# Filter the dataframe for 'Primary_Payer_Class_Cd' not equal to 'P'
df_other = df[df['Primary_Payer_Class_Cd'] != 'P']
diagnosis_counts_other = df_other['Diag1'].value_counts().head(10)
# Create the bar graph
fig = go.Figure()
# Add the bar for 'Primary_Payer_Class_Cd' equals 'P'
fig.add_trace(go.Bar(x=diagnosis_counts_p.index, y=diagnosis_counts_p.values, name="Self-Pay"))
# Add the bar for 'Primary_Payer_Class_Cd' not equal to 'P'
fig.add_trace(go.Bar(x=diagnosis_counts_other.index, y=diagnosis_counts_other.values, name="Other Payer Classes"))
# Customize the layout
fig.update_layout(
    title="Top 10 Diagnoses by Payer Class",
    xaxis_title="Diagnosis",
    yaxis_title="Count"
)
# Show the graph
fig.show()

In [ ]:

# Calculate the total count of diagnoses
total_counts = df['Diag1'].value_counts().head(10)
# Filter the dataframe for 'Primary_Payer_Class_Cd' equals 'P'
df_p = df[df['Primary_Payer_Class_Cd'] == 'P']
diagnosis_counts_p = df_p['Diag1'].value_counts().head(10)
# Calculate the percentage for 'Primary_Payer_Class_Cd' equals 'P'
percentages_p = (diagnosis_counts_p / total_counts) * 100
# Calculate the percentage for 'Primary_Payer_Class_Cd' not equal to 'P'
percentages_other = 100 - percentages_p
# Create the bar graph
fig = go.Figure()
# Add the bar for 'Primary_Payer_Class_Cd' equals 'P'
fig.add_trace(go.Bar(x=percentages_p.index, y=percentages_p.values, name="Self-Pay"))
# Add the bar for 'Primary_Payer_Class_Cd' not equal to 'P'
fig.add_trace(go.Bar(x=percentages_other.index, y=percentages_other.values, name="Other Payer Classes"))
# Customize the layout
fig.update_layout(
    title="Percentage of Payer Class by Diagnosis",
    xaxis_title="Diagnosis",
    yaxis_title="Percentage",
    yaxis_tickformat=".1f"
)
# Show the graph
fig.show()

In [ ]:
#This should be the correct one

# Get the top 10 diagnoses
top_diagnoses = df['Diag1'].value_counts().head(10).index
# Initialize lists to store the percentages
percentages_p = []
percentages_other = []
# Calculate the percentage for each diagnosis
for diagnosis in top_diagnoses:
    total_count = df[df['Diag1'] == diagnosis].shape[0]
    p_count = df[(df['Diag1'] == diagnosis) & (df['Primary_Payer_Class_Cd'] == 'P')].shape[0]
    p_percentage = (p_count / total_count) * 100
    percentages_p.append(p_percentage)
    percentages_other.append(100 - p_percentage)
# Create the bar graph
fig = go.Figure()
# Add the bar for 'Primary_Payer_Class_Cd' equals 'P'
fig.add_trace(go.Bar(x=top_diagnoses, y=percentages_p, name="Self-Pay"))
# Add the bar for 'Primary_Payer_Class_Cd' not equal to 'P'
fig.add_trace(go.Bar(x=top_diagnoses, y=percentages_other, name="Other Payer Classes"))
# Customize the layout
fig.update_layout(
    title="Percentage of Payer Class by Diagnosis",
    xaxis_title="Diagnosis",
    yaxis_title="Percentage",
    yaxis_tickformat=".1f"
)
# Show the graph
fig.show()

In [ ]:

# Get the top 10 diagnoses based on frequency in the diag1 column
top_10_diagnoses = df['Diag1'].value_counts().nlargest(10).index
# Define the age ranges
age_ranges = [(0, 17), (18, 25), (26, 35), (36, 50), (51, 70), (71, float('inf'))]
# Categorize age into the defined ranges
df['Age Range'] = pd.cut(df['Age'], bins=[range_[0] - 0.5 for range_ in age_ranges] + [age_ranges[-1][1] + 0.5],
                         labels=[f'{range_[0]}-{range_[1]}' for range_ in age_ranges])
# Filter the DataFrame for the top 10 diagnoses and age ranges
filtered_df = df[df['Diag1'].isin(top_10_diagnoses)]
# Create an interactive visualization using Plotly
fig = px.histogram(filtered_df, x='Diag1', color='Age Range', title='Top 10 Diagnoses by Age Range',
                   labels={'Diag1': 'Diagnosis', 'Age Range': 'Age Range'},
                   category_orders={'Diag1': top_10_diagnoses, 'Age Range': [f'{range_[0]}-{range_[1]}' for range_ in age_ranges]})
fig.show()

In [ ]:

# Get the top 10 diagnoses based on frequency in the diag1 column
top_10_diagnoses = df['Diag1'].value_counts().nlargest(10).index
# Define the age ranges
age_ranges = [(0, 17), (18, 25), (26, 35), (36, 50), (51, 70), (71, float('inf'))]
# Categorize age into the defined ranges
df['Age Range'] = pd.cut(df['Age'], bins=[range_[0] - 0.5 for range_ in age_ranges] + [age_ranges[-1][1] + 0.5],
                         labels=[f'{range_[0]}-{range_[1]}' for range_ in age_ranges])
# Filter the DataFrame for the top 10 diagnoses and age ranges
filtered_df = df[df['Diag1'].isin(top_10_diagnoses)]
# Create an interactive visualization using Plotly
fig = px.histogram(filtered_df, y='Age Range', color='Diag1', title='Age Range Distribution for Top 10 Diagnoses',
                   labels={'Age Range': 'Age Range', 'Diag1': 'Diagnosis'},
                   category_orders={'Age Range': [f'{range_[0]}-{range_[1]}' for range_ in age_ranges], 'Diag1': top_10_diagnoses})
fig.update_layout(barmode='stack')  # Display bars stacked on top of each other
fig.show()

In [ ]:

# Get the top 10 diagnoses based on frequency in diag1 column
top_10_diagnoses = df['Diag1'].value_counts().nlargest(10).index
# Define the age ranges
age_ranges = [(0, 17), (18, 25), (26, 35), (36, 50), (51, 70), (71, float('inf'))]
# Create a new column 'Age Range' based on the age ranges
df['Age Range'] = pd.cut(df['Age'], bins=[range[0] - 0.5 for range in age_ranges] + [age_ranges[-1][1] + 0.5],
                         labels=['{}-{}'.format(range[0], range[1]) for range in age_ranges])
# Filter the DataFrame for the top 10 diagnoses
filtered_df = df[df['Diag1'].isin(top_10_diagnoses)]
# Create an interactive visualization using Plotly
fig = px.histogram(filtered_df, x='Age Range', color='Diag1', title='Top 10 Diagnoses by Age Range',
                   labels={'Age Range': 'Age Range', 'count': 'Count'})
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': [range[0] for range in age_ranges]},
                  barmode='group')
fig.show()

In [ ]:
# Define the age groups
age_groups = [(18, 32), (33, 48), (49, 64)]
# Create a new column 'AgeGroup' in the dataframe based on age ranges
df['AgeGroup'] = pd.cut(df['Age'], bins=[age[0] for age in age_groups] + [age_groups[-1][1]], labels=[f"{age[0]}-{age[1]}" for age in age_groups])
# Group the counts of diagnoses by age group and diagnosis, and select the top 10 for each group
diagnosis_counts = df.groupby(['AgeGroup', 'Diag1']).size().reset_index(name='Count')
diagnosis_counts = diagnosis_counts.groupby('AgeGroup').apply(lambda x: x.nlargest(10, 'Count')).reset_index(drop=True)
# Create the bar graph using Plotly
fig = px.bar(diagnosis_counts, x='AgeGroup', y='Count', color='Diag1', title='Top 10 Diagnoses for Each Age Group')
fig.update_layout(barmode='group')
fig.show()

In [ ]:
# Define the age groups
age_groups = [(18, 32), (33, 48), (49, 64)]
# Create a new column 'AgeGroup' in the dataframe based on age ranges
df['AgeGroup'] = pd.cut(df['Age'], bins=[age[0] for age in age_groups] + [age_groups[-1][1]],
                        labels=[f"{age[0]}-{age[1]}" for age in age_groups])
# Create a dictionary to map ICD codes to diagnosis names
diagnosis_names = {
    "R109": "Abdominal pain",
    "R112": "Nausea with vomiting",
    "N390": "UTI",
    "R079": "Chest pain",
    "M545": "Low back pain",
    "J069": "Acute URI",
    "R0789": "Other chest pain",
    "R51": "Headache",
    "J209": "Acute bronchitis",
    "A419": "Sepsis",
    "K047": "Periapical abscess",
    "K029": "Dental caries",
    "G43909": "Migraine",
    "Z1211": "Colon screening",
    "J441": "COPD",
    "E11621": "Type 2 diabetes"
}
# Replace the ICD codes with diagnosis names in the dataframe
df['Diag1'] = df['Diag1'].map(diagnosis_names)
# Group the counts of diagnoses by age group and diagnosis, and select the top 10 for each group
diagnosis_counts = df.groupby(['AgeGroup', 'Diag1']).size().reset_index(name='Count')
diagnosis_counts = diagnosis_counts.groupby('AgeGroup').apply(lambda x: x.nlargest(10, 'Count')).reset_index(drop=True)
# Create the bar graph using Plotly
fig = px.bar(diagnosis_counts, x='AgeGroup', y='Count', color='Diag1', title='Top 10 Diagnoses by Age Group')
fig.update_layout(barmode='group')


fig.show()

In [ ]:
ICD10_legend2 = {"R109":"Abdominal pain","R112":"Nausea with vomiting", 
                "N390":"UTI", "R079":"Chest pain","M545":"Low back pain",
               "J069":"Acute URI", "R0789":"Other chest pain",
               "R51":"Headache","J209":"Acute bronchitis","A419":"Sepsis",
                "K047":"Periapical abscess","K029":"Dental caries",
                "G43909":"Migraine","Z1211":"Colon screening","J441":"COPD",
                "E11621":"Type 2 diabetes"}

In [ ]:

# Get the top 10 diagnoses based on frequency in the diag1 column
top_10_diagnoses = df['Diag1'].value_counts().nlargest(10).index
# Define the age ranges
age_ranges = [(18,32), (33, 48), (49, 64)]
# Categorize age into the defined ranges
df['Age Range'] = pd.cut(df['Age'], bins=[range_[0] - 0.5 for range_ in age_ranges] + [age_ranges[-1][1] + 0.5],
                         labels=[f'{range_[0]}-{range_[1]}' for range_ in age_ranges])
# Filter the DataFrame for the top 10 diagnoses and age ranges
filtered_df = df[df['Diag1'].isin(top_10_diagnoses)]
# Create an interactive visualization using Plotly
fig = px.histogram(filtered_df, x='Diag1', color='Age Range', title='Overall Top 10 Diagnoses by Age Range',
                   labels={'Diag1': 'Diagnosis', 'Age Range': 'Age Range'},
                   category_orders={'Diag1': top_10_diagnoses, 'Age Range': [f'{range_[0]}-{range_[1]}' for range_ in age_ranges]})
fig.update_layout(barmode='group')  # Display bars stacked on top of each other
fig.show()

In [ ]:
print(df['Hospital_ID'].value_counts())

In [ ]:
print(df['Patient_Zip'].value_counts())

## Graphs using full dataset

In [ ]:
%%time
query='''
select sewanee_health_data."File_Type" , sewanee_health_data."Data_Yr", sewanee_health_data."Hospital_ID", sewanee_health_data."Patient_Zip", sewanee_health_data."Patient_Sex", sewanee_health_data."Admit_Hr", sewanee_health_data."Admission_Type", sewanee_health_data."Admission_Source", sewanee_health_data."Accident_Code", sewanee_health_data."Primary_Health_Plan_Id",
sewanee_health_data."Diag1", sewanee_health_data."Diag2", sewanee_health_data."Diag3", sewanee_health_data."Diag4", sewanee_health_data."Diag5", sewanee_health_data."Diag6", sewanee_health_data."Diag7", sewanee_health_data."Diag8", sewanee_health_data."Diag9", sewanee_health_data."Diag10", sewanee_health_data."Diag11", sewanee_health_data."Diag12", sewanee_health_data."Diag13", sewanee_health_data."Diag14", sewanee_health_data."Diag5", sewanee_health_data."Diag16", sewanee_health_data."Diag17", sewanee_health_data."Diag18", 
sewanee_health_data."Patient_Reason_Visit1", sewanee_health_data."Patient_Reason_Visit2", sewanee_health_data."Patient_Reason_Visit3", sewanee_health_data."Patient_Race_Ethnicity", sewanee_health_data."Type_ER_Visit", sewanee_health_data."Patient_ID", sewanee_health_data."Age", sewanee_health_data."Hospital_Id_JAR", sewanee_health_data."Primary_Payer_Class_Cd"
 from sewanee_health_data
where sewanee_health_data."Patient_Zip" in ('37710',
 '37938',
 '37769',
 '37717',
 '37828',
 '37705',
 '37931',
 '37849',
 '37716',
 '37754',
 '37840',
 '37830',
 '37831',
 '37360',
 '37160',
 '37334',
 '37018',
 '37034',
 '37180',
 '37183',
 '37060',
 '37388',
 '37144',
 '37161',
 '37091',
 '37020',
 '37153',
 '38221',
 '38341',
 '38387',
 '38318',
 '38333',
 '38320',
 '38317',
 '38380',
 '37379',
 '37327',
 '37338',
 '37381',
 '37332',
 '37367',
 '37321',
 '38572',
 '37337',
 '37886',
 '37920',
 '37801',
 '37853',
 '37802',
 '37772',
 '37865',
 '37882',
 '37737',
 '37701',
 '37804',
 '37803',
 '37742',
 '37777',
 '37878',
 '37310',
 '37353',
 '37323',
 '37364',
 '37320',
 '37336',
 '37362',
 '37311',
 '37312',
 '37870',
 '37729',
 '37819',
 '37715',
 '37847',
 '37757',
 '37766',
 '37762',
 '37756',
 '37714',
 '37016',
 '37149',
 '37190',
 '37037',
 '37118',
 '37026',
 '37095',
 '37110',
 '37166',
 '37357',
 '38220',
 '38305',
 '38235',
 '38321',
 '38390',
 '38388',
 '38342',
 '38201',
 '38348',
 '38358',
 '38324',
 '38344',
 '38258',
 '37601',
 '37658',
 '37682',
 '37687',
 '37640',
 '37604',
 '37683',
 '37643',
 '37694',
 '37035',
 '37187',
 '37036',
 '37082',
 '37080',
 '37143',
 '37209',
 '37062',
 '37015',
 '37043',
 '37032',
 '37146',
 '38352',
 '38315',
 '38356',
 '38359',
 '38345',
 '38366',
 '38347',
 '38334',
 '38368',
 '38313',
 '38340',
 '38332',
 '38044',
 '37724',
 '37707',
 '37879',
 '37851',
 '37869',
 '37825',
 '37752',
 '37730',
 '37881',
 '38575',
 '38588',
 '38573',
 '38541',
 '37150',
 '38551',
 '38568',
 '37722',
 '37753',
 '37743',
 '37821',
 '37713',
 '37843',
 '37727',
 '37389',
 '37330',
 '37355',
 '37342',
 '37382',
 '38001',
 '38050',
 '38021',
 '38336',
 '38040',
 '38337',
 '38343',
 '38006',
 '38034',
 '37723',
 '38583',
 '38557',
 '37854',
 '38558',
 '38555',
 '38578',
 '38574',
 '38571',
 '37217',
 '37210',
 '37070',
 '37222',
 '37064',
 '37073',
 '37189',
 '37211',
 '37212',
 '37218',
 '37229',
 '37135',
 '37086',
 '37122',
 '37240',
 '37201',
 '37203',
 '37221',
 '37224',
 '37250',
 '37234',
 '37138',
 '37215',
 '37228',
 '37076',
 '37072',
 '37027',
 '37238',
 '37235',
 '37246',
 '37219',
 '37204',
 '37214',
 '37207',
 '37205',
 '37216',
 '37213',
 '37202',
 '37236',
 '37013',
 '37206',
 '37208',
 '37116',
 '37220',
 '37024',
 '37243',
 '37242',
 '37227',
 '37115',
 '37011',
 '37232',
 '38311',
 '38374',
 '38363',
 '38329',
 '38371',
 '38328',
 '38582',
 '37012',
 '38548',
 '38567',
 '38544',
 '38581',
 '37059',
 '38569',
 '37029',
 '37052',
 '37101',
 '37165',
 '37055',
 '37181',
 '37051',
 '37056',
 '37025',
 '37171',
 '38259',
 '38240',
 '38047',
 '38024',
 '38080',
 '38025',
 '38030',
 '38059',
 '38007',
 '38070',
 '38017',
 '38046',
 '38057',
 '38060',
 '38039',
 '38049',
 '38028',
 '38069',
 '38045',
 '38002',
 '38068',
 '38066',
 '38036',
 '38075',
 '38076',
 '38042',
 '38048',
 '38589',
 '38504',
 '38565',
 '37726',
 '38554',
 '38549',
 '38577',
 '38556',
 '38553',
 '37380',
 '37375',
 '37306',
 '37366',
 '37356',
 '37335',
 '37352',
 '37324',
 '37376',
 '37383',
 '37398',
 '37345',
 '37318',
 '37328',
 '38316',
 '38382',
 '38355',
 '38338',
 '38346',
 '38330',
 '38233',
 '38331',
 '38369',
 '38389',
 '38478',
 '38472',
 '38451',
 '37047',
 '38477',
 '38468',
 '38453',
 '38459',
 '38455',
 '38473',
 '38460',
 '38483',
 '38449',
 '38464',
 '38474',
 '38456',
 '37861',
 '37709',
 '37848',
 '37811',
 '37888',
 '37708',
 '37806',
 '37779',
 '37818',
 '37711',
 '37745',
 '37616',
 '37656',
 '37809',
 '37681',
 '37810',
 '37641',
 '37857',
 '37374',
 '37339',
 '37313',
 '37301',
 '37305',
 '37387',
 '37365',
 '37860',
 '37890',
 '37815',
 '37816',
 '37877',
 '37813',
 '37814',
 '37891',
 '37778',
 '37343',
 '37377',
 '37384',
 '37403',
 '37409',
 '37401',
 '37304',
 '37412',
 '37363',
 '37373',
 '37406',
 '37408',
 '37341',
 '37414',
 '37421',
 '37405',
 '37308',
 '37402',
 '37415',
 '37404',
 '37419',
 '37424',
 '37422',
 '37351',
 '37407',
 '37410',
 '37350',
 '37315',
 '37411',
 '37416',
 '37450',
 '37302',
 '37731',
 '37765',
 '38067',
 '38061',
 '38008',
 '38392',
 '38052',
 '38381',
 '38370',
 '38326',
 '38379',
 '38357',
 '38310',
 '38376',
 '38372',
 '38327',
 '38475',
 '38361',
 '38425',
 '38365',
 '37642',
 '37660',
 '37645',
 '37873',
 '38301',
 '38063',
 '38012',
 '38037',
 '38351',
 '38251',
 '38242',
 '38223',
 '38222',
 '38229',
 '38224',
 '38256',
 '38236',
 '38231',
 '37033',
 '37140',
 '38461',
 '38487',
 '37137',
 '38462',
 '37098',
 '38454',
 '38476',
 '37185',
 '37061',
 '37175',
 '37050',
 '37178',
 '37078',
 '37097',
 '37134',
 '38564',
 '37030',
 '37145',
 '38501',
 '38562',
 '38545',
 '38506',
 '37820',
 '37871',
 '37725',
 '37764',
 '37760',
 '37876',
 '37691',
 '37688',
 '37680',
 '37934',
 '37919',
 '37933',
 '37928',
 '37922',
 '37721',
 '37930',
 '37901',
 '37997',
 '37771',
 '37924',
 '37909',
 '37917',
 '37902',
 '37916',
 '37996',
 '37940',
 '37998',
 '37912',
 '37932',
 '37939',
 '37914',
 '37807',
 '37915',
 '37918',
 '37923',
 '37950',
 '37927',
 '37995',
 '37929',
 '37921',
 '38079',
 '38077',
 '38041',
 '38463',
 '38457',
 '38469',
 '38481',
 '38486',
 '38485',
 '37096',
 '37359',
 '37348',
 '38488',
 '37774',
 '37846',
 '37874',
 '37885',
 '37371',
 '37329',
 '37303',
 '37354',
 '37309',
 '37880',
 '37322',
 '37370',
 '37325',
 '37331',
 '37826',
 '38375',
 '38393',
 '38339',
 '38367',
 '37074',
 '37022',
 '37186',
 '37057',
 '37083',
 '38302',
 '38362',
 '38391',
 '38308',
 '38378',
 '38303',
 '37397',
 '37396',
 '37347',
 '37340',
 '37174',
 '37019',
 '37046',
 '38401',
 '38482',
 '38402',
 '37369',
 '37314',
 '37385',
 '37142',
 '37010',
 '37044',
 '37079',
 '37041',
 '37042',
 '37191',
 '37040',
 '42223',
 '37829',
 '37845',
 '37887',
 '37748',
 '37872',
 '37770',
 '37733',
 '37719',
 '37852',
 '38232',
 '38271',
 '38281',
 '38253',
 '38257',
 '38260',
 '38237',
 '38255',
 '38254',
 '38261',
 '38543',
 '38570',
 '38542',
 '38580',
 '37307',
 '37326',
 '37333',
 '37391',
 '37317',
 '37361',
 '38502',
 '38503',
 '37763',
 '37148',
 '37172',
 '37048',
 '37141',
 '37188',
 '37152',
 '37049',
 '37090',
 '37063',
 '37130',
 '37129',
 '37133',
 '37089',
 '37127',
 '37167',
 '37085',
 '37014',
 '37131',
 '37132',
 '37128',
 '37841',
 '37755',
 '37732',
 '37892',
 '37862',
 '37863',
 '37864',
 '37738',
 '37868',
 '38109',
 '38132',
 '38124',
 '38125',
 '38101',
 '38137',
 '38139',
 '38157',
 '38182',
 '38152',
 '38186',
 '38113',
 '38193',
 '38016',
 '38104',
 '38112',
 '38103',
 '38114',
 '38126',
 '38138',
 '38177',
 '38173',
 '38166',
 '38117',
 '38128',
 '38133',
 '38011',
 '38130',
 '38168',
 '38187',
 '38197',
 '38161',
 '38118',
 '38111',
 '38107',
 '38088',
 '38188',
 '38120',
 '38053',
 '38106',
 '38127',
 '38134',
 '38175',
 '38115',
 '38105',
 '38174',
 '38167',
 '37501',
 '38018',
 '38004',
 '38116',
 '38119',
 '38184',
 '38141',
 '38190',
 '38181',
 '38131',
 '38055',
 '38108',
 '38122',
 '38135',
 '38183',
 '38027',
 '38029',
 '38083',
 '38014',
 '38054',
 '38136',
 '38547',
 '37087',
 '38563',
 '37184',
 '38552',
 '38560',
 '37151',
 '37023',
 '37058',
 '37028',
 '37662',
 '37659',
 '37620',
 '37665',
 '37615',
 '37664',
 '37699',
 '37618',
 '37686',
 '37621',
 '37625',
 '37617',
 '37663',
 '37669',
 '37066',
 '37077',
 '37031',
 '37075',
 '37119',
 '38023',
 '38058',
 '38019',
 '38071',
 '38015',
 '37692',
 '37657',
 '37650',
 '37866',
 '38559',
 '38585',
 '37378',
 '37394',
 '38550',
 '37602',
 '37614',
 '37684',
 '37605',
 '37690',
 '38450',
 '38471',
 '38452',
 '38241',
 '38225',
 '38226',
 '38238',
 '38230',
 '38579',
 '38587',
 '37179',
 '37068',
 '37069',
 '37067',
 '37065',
 '37071',
 '37121',
 '37088')
'''
TN_data= pd.read_sql(query, connection)

#Change data so that it incorporates all zip codes in TN to make the rural vs urban graphs

In [ ]:
df1 = pd.read_csv('ZIP-COUNTY-TN.csv')
TN_data['Patient_Zip']=TN_data['Patient_Zip'].astype(int)
df2 = pd.merge(TN_data,df1,left_on='Patient_Zip',right_on='ZIP')
df2['COUNTYNAME']=df2['COUNTYNAME'].apply(lambda x: " ".join(x.split(" ")[:-1]))
l='Bedford,Benton,Bledsoe,Campbell,Cannon,Carroll,Cheatham,Chester,Claiborne,Clay,Cocke,Coffee,Crockett,Cumberland,Decatur,DeKalb,Dickson,Dyer,Fentress,Franklin,Gibson,Giles,Grainger,Greene,Grundy,Hancock,Hardeman,Hardin,Haywood,Henderson,Henry,Hickman,Houston,Humphreys,Jackson,Johnson,Lake,Lauderdale,Lawrence,Lewis,Lincoln,Macon,Marion,Marshall,Maury,McMinn,McNairy,Meigs,Monroe,Moore,Obion,Overton,Perry,Pickett,Polk,Putnam,Rhea,Scott,Sevier,Smith,Stewart,Tipton,Trousdale,Union,Van,Buren,Warren,Wayne,Weakley,White'.split(',')
df2['U/R']=df2['COUNTYNAME'].apply(lambda x: 'R' if x in l else 'U' )
df2

In [ ]:
df2.Patient_Zip.nunique()

In [ ]:
df2.info()

In [ ]:
df_filtered = df2[df2['U/R'] == 'R']
top_10_diagnoses = df_filtered['Diag1'].value_counts().nlargest(10)
# Creating the bar graph using Plotly
fig = px.bar(x=top_10_diagnoses.index, y=top_10_diagnoses.values, labels={'x': 'Diagnosis', 'y': 'Count'})
fig.update_layout(title="Top 10 Diagnoses for Rural TN",
                  xaxis_title="Diagnosis",
                  yaxis_title="Count")
fig.show()

In [ ]:
df_filtered = df2[df2['U/R'] == 'U']
top_10_diagnoses = df_filtered['Diag1'].value_counts().nlargest(10)
# Creating the bar graph using Plotly
fig = px.bar(x=top_10_diagnoses.index, y=top_10_diagnoses.values, labels={'x': 'Diagnosis', 'y': 'Count'})
fig.update_layout(title="Top 10 Diagnoses for Urban TN",
                  xaxis_title="Diagnosis",
                  yaxis_title="Count")
fig.show()

In [ ]:
df_filtered = df2[df2['U/R'] == 'R']
total_population = len(df_filtered)
diagnosis_counts = df_filtered['Diag1'].value_counts()
top_10_diagnoses = diagnosis_counts.nlargest(10)
diagnosis_percentages = (top_10_diagnoses / total_population) * 100
# Creating the bar graph using Plotly
fig = px.bar(x=diagnosis_percentages.index, y=diagnosis_percentages.values, labels={'x': 'Diagnosis', 'y': 'Percentage'})
fig.update_layout(title="Top 10 Diagnoses Percentage for Rural TN",
                  xaxis_title="Diagnosis",
                  yaxis_title="Percentage")
fig.show()

In [ ]:
df_filtered = df2[df2['U/R'] == 'U']
total_population = len(df_filtered)
diagnosis_counts = df_filtered['Diag1'].value_counts()
top_10_diagnoses = diagnosis_counts.nlargest(10)
diagnosis_percentages = (top_10_diagnoses / total_population) * 100
# Creating the bar graph using Plotly
fig = px.bar(x=diagnosis_percentages.index, y=diagnosis_percentages.values, labels={'x': 'Diagnosis', 'y': 'Percentage'})
fig.update_layout(title="Top 10 Diagnoses Percentage for Urban TN",
                  xaxis_title="Diagnosis",
                  yaxis_title="Percentage")
fig.show()

In [ ]:

# Get the top 10 diagnoses
top_diagnoses = df2['Diag1'].value_counts().head(10).index
# Initialize lists to store the percentages
percentages_p = []
percentages_other = []
# Calculate the percentage for each diagnosis
for diagnosis in top_diagnoses:
    total_count = df2[df2['Diag1'] == diagnosis].shape[0]
    p_count = df2[(df2['Diag1'] == diagnosis) & (df2['Primary_Payer_Class_Cd'] == 'P')].shape[0]
    p_percentage = (p_count / total_count) * 100
    percentages_p.append(p_percentage)
    percentages_other.append(100 - p_percentage)
# Create the bar graph
fig = go.Figure()
# Add the bar for 'Primary_Payer_Class_Cd' equals 'P'
fig.add_trace(go.Bar(x=top_diagnoses, y=percentages_p, name="Self-Pay"))
# Add the bar for 'Primary_Payer_Class_Cd' not equal to 'P'
fig.add_trace(go.Bar(x=top_diagnoses, y=percentages_other, name="Other Payer Classes"))
# Customize the layout
fig.update_layout(
    title="Percentage of Payer Class by Diagnosis",
    xaxis_title="Diagnosis",
    yaxis_title="Percentage",
    yaxis_tickformat=".1f"
)
# Show the graph
fig.show()

In [ ]:
ICD10_legend3 = {"R109":"Abdominal pain",
                "N390":"UTI", "R079":"Chest pain","M545":"Low back pain",
               "J069":"Acute URI", "R0789":"Other chest pain",
               "R51":"Headache","A419":"Sepsis",
                "Z3800":"Liveborn infant","Z1211":"Colon screening"}

In [ ]:

# Get the top 10 diagnoses
top_diagnoses = df2['Diag1'].value_counts().head(10).index
# Create a dictionary to map diagnosis codes to diagnosis names
diagnosis_names = {
    "R109": "Abdominal pain",
    "N390": "UTI",
    "R079": "Chest pain",
    "M545": "Low back pain",
    "J069": "Acute URI",
    "R0789": "Other chest pain",
    "R51": "Headache",
    "A419": "Sepsis",
    "Z3800": "Liveborn infant",
    "Z1211": "Colon screening"
}
# Initialize lists to store the percentages
percentages_p = []
percentages_other = []
# Calculate the percentage for each diagnosis
for diagnosis in top_diagnoses:
    total_count = df2[df2['Diag1'] == diagnosis].shape[0]
    p_count = df2[(df2['Diag1'] == diagnosis) & (df2['Primary_Payer_Class_Cd'] == 'P')].shape[0]
    p_percentage = (p_count / total_count) * 100
    percentages_p.append(p_percentage)
    percentages_other.append(100 - p_percentage)
# Create the bar graph
fig = go.Figure()
# Add the bar for 'Primary_Payer_Class_Cd' equals 'P'
fig.add_trace(go.Bar(x=[diagnosis_names[diag] for diag in top_diagnoses], y=percentages_p, name="Self-Pay"))
# Add the bar for 'Primary_Payer_Class_Cd' not equal to 'P'
fig.add_trace(go.Bar(x=[diagnosis_names[diag] for diag in top_diagnoses], y=percentages_other, name="Other Payer Classes"))
# Customize the layout
fig.update_layout(
    title="Percentage of Payer Class by Diagnosis",
    xaxis_title="Diagnosis",
    yaxis_title="Percentage",
    yaxis_tickformat=".1f"
)
# Show the graph
fig.show()

## TDH Data Agreement Testing & Cleaning

In [ ]:
values = ['E10.9']
count = len(df2[df2['Diag1'].isin(values)])
print(count)

In [ ]:
# Finding individual Diag1 values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df['Diag1'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding individual Diag1 values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df2['Diag1'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding individual Primary_Payer_Class_Cd values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df['Primary_Payer_Class_Cd'].value_counts().items():
    if n < 11:
        print(f"Before Change Value: {value}, & Count: {n}")
        df.loc[df['Primary_Payer_Class_Cd'] == value, 'Primary_Payer_Class_Cd'] = 'Other'
        
for value, n in df['Primary_Payer_Class_Cd'].value_counts().items():
    if n < 11:
        print(f"After Change Value: {value}, & Count: {n}")
        
# With our current data, the new Other category has less than 11 counts so we will be removing the corrosponding rows
for value, n in df['Primary_Payer_Class_Cd'].value_counts().items():
    if n < 11:
        df = df[df['Primary_Payer_Class_Cd'] != value]

# Double checking that there are no remaining individual counts less than 11
for value, n in df['Primary_Payer_Class_Cd'].value_counts().items():
    if n < 11:
        print(f"After Removal Value: {value}, & Count: {n}")

In [ ]:
# Finding individual Primary_Payer_Class_Cd values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df2['Primary_Payer_Class_Cd'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding individual Patient_Zip values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df['Patient_Zip'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")
        # To prevent errors when displaying this data on a map, we are removing these zipcodes rather than adding an "Other" value
        df = df[df['Patient_Zip'] != value]

# Double checking
for value, n in df['Patient_Zip'].value_counts().items():
    if n < 11:
        print(f"Double Checked Value: {value}, Count: {n}")

In [ ]:
# Finding individual Patient_Zip values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df2['Patient_Zip'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding individual Age values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df['Age'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding individual Age values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df2['Age'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding individual U/R values less than 11 counts to suppress or unidentify in the future per TDH agreement
for value, n in df2['U/R'].value_counts().items():
    if n < 11:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding the amount of people 75+ to aggregate in the future per TDH agreement
for value, n in df['Age'].value_counts().items():
    if value >= 75:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding the amount of people 75+ to aggregate in the future per TDH agreement
for value, n in df2['Age'].value_counts().items():
    if value >= 75:
        print(f"Value: {value}, Count: {n}")

In [ ]:
# Finding geographic units with populations less than 20,000 to suppress or unidentify in the future per TDH agreement


In [ ]:
df2.Diag1.unique()

In [ ]:
values = ['I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19']
count = len(df2[df2['Diag1'].isin(values)])
print(count)

In [ ]:
values = ['I20', 'I24.0', 'I24.8', 'I24.9', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L98.0', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I11.0', 'J81.0', 'R56', 'E86', 'E10.9', 'E11.9', 'E10.1', 'E13.1', 'E11.0', 'E13.0', 'E10.641', 'E11.641', 'E10.6', 'E11.6', 'E10.8', 'E11.8', 'K52.9', 'K52.89', 'G40', 'I10', 'I11.9', 'E16.2', 'N10', 'N11', 'N12', 'N70', 'N73', 'A15.0', 'A15.5', 'A15.9', 'H66', 'J02', 'J03', 'J06', 'J31.2', 'A15.4', 'A15.6', 'A15.8', 'A17', 'A18', 'A19']
count = len(df2[df2['Diag1'].isin(values)])
print(count)

In [ ]:
values = ['E109']
count = len(df2[df2['Diag1'].isin(values)])
print(count)

## Graphs using full dataset continued

In [ ]:
# Specify the zip codes and their respective counties
zip_county_mapping = {
    '37318': 'Franklin County',
    '37398': 'Franklin County',
    '37330': 'Franklin County',
    '37306': 'Franklin County',
    '37345': 'Franklin County',
    '37324': 'Franklin County',
    '37375': 'Franklin County',
    '37376': 'Franklin County',
    '37355': 'Coffee County',
    '37388': 'Coffee County',
    '37342': 'Coffee County',
    '37349': 'Coffee County',
    '37352': 'Moore County'
}
# Specify the preventable diagnoses
preventable_diagnoses = [
    'I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19'
]
# Filter the DataFrame based on zip codes and preventable diagnoses
filtered_data = df2[df2['Patient_Zip'].isin(zip_county_mapping.keys()) & df2['Diag1'].isin(preventable_diagnoses)]
# Assign the respective counties to each zip code
filtered_data['County'] = filtered_data['Patient_Zip'].map(zip_county_mapping)
# Create an interactive histogram
fig = px.histogram(filtered_data, x='County', title='Count of Counties for Preventable Diagnoses')
# Display the interactive histogram
fig.show()

In [ ]:
#Add additional counties/zipcodes
zip_county_mapping = {
    '37318': 'Franklin County',
    '37398': 'Franklin County',
    '37330': 'Franklin County',
    '37306': 'Franklin County',
    '37345': 'Franklin County',
    '37324': 'Franklin County',
    '37375': 'Franklin County',
    '37376': 'Franklin County',
    '37383': 'Franklin County',
    '37355': 'Coffee County',
    '37388': 'Coffee County',
    '37342': 'Coffee County',
    '37349': 'Coffee County',
    '37382': 'Coffee County',
    '37389': 'Coffee County',
    '37018': 'Coffee County',
    '37361': 'Coffee County',
    '37352': 'Moore County',
    '37020': 'Bedford County',
    '37160': 'Bedford County',
    '37161': 'Bedford County',
    '37162': 'Bedford County',
    '37180': 'Bedford County',
    '37183': 'Bedford County',
    '37110': 'Warren County',
    '37111': 'Warren County',
    '37357': 'Warren County',
    '37378': 'Warren County',
    '37394': 'Warren County',
    '38550': 'Warren County',
    '38581': 'Warren County',
    '37016': 'Cannon County',
    '37026': 'Cannon County',
    '37190': 'Cannon County',
    '37301': 'Grundy County',
    '37305': 'Grundy County',
    '37313': 'Grundy County',
    '37339': 'Grundy County',
    '37356': 'Grundy County',
    '37365': 'Grundy County',
    '37366': 'Grundy County',
    '37387': 'Grundy County',
    '37340': 'Marion County',
    '37347': 'Marion County',
    '37374': 'Marion County',
    '37379': 'Marion County',
    '37396': 'Marion County',
    '37397': 'Marion County'
}
preventable_diagnoses = [
    'I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04',
    'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109',
    'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40',
    'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06',
    'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19'
]
filtered_data = df2[df2['Patient_Zip'].isin(zip_county_mapping.keys()) & df2['Diag1'].isin(preventable_diagnoses)]
filtered_data['County'] = filtered_data['Patient_Zip'].map(zip_county_mapping)
fig = px.histogram(filtered_data, x='County', title='Count of Counties for Preventable Diagnoses')
fig.show()